In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
# Definindo o tamanho do chunk para x linhas
chunksize = 10
reader = pd.read_csv('Quali_20240601.csv', chunksize=chunksize)

# Pegando o primeiro chunk
chunk = next(reader)

# Exibindo o chunk
print(chunk)

                    Legal Fund Name Including Umbrella Fund Code Citi Code  \
0                                                 C 11     00F9A      009A   
1                                                 C 11     00F9A      009A   
2                                             R-VIP 50     00F9G      009G   
3                                             R-VIP 50     00F9G      009G   
4                                            R-VIP 100     00F9I      009I   
..                                                 ...       ...       ...   
995  AMUNDI FUNDS - AMUNDI FUNDS ABSOLUTE RETURN EU...     04F2V      042V   
996  AMUNDI FUNDS - AMUNDI FUNDS ABSOLUTE RETURN EU...     04F2V      042V   
997  AMUNDI FUNDS - AMUNDI FUNDS ABSOLUTE RETURN EU...     04F2V      042V   
998  AMUNDI FUNDS - AMUNDI FUNDS ABSOLUTE RETURN EU...     04F2V      042V   
999  AMUNDI FUNDS - AMUNDI FUNDS ABSOLUTE RETURN EU...     04F2V      042V   

             ISIN     WKN Apir Code    SEDOL  As Of Date  ESG S

C:\Users\Vini\AppData\Local\Temp\ipykernel_13644\3577502960.py:6: DtypeWarning: Columns (5,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,107,111,112,113,114,115,116,117,118,120,121,122,124,125,127,138,139,140,142,149,158,159,160,163,164,173,174,175,176,177,178,181,182,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,264,268,272,276,280,284,288,292,296,300,304,308,312,316,320,324,328,332,336,340,344,348,352,356,360,364,368,372,376,380,384,388,392,396,400,404,408,412,416,420,424,428,432,436,440,444,448,452,456,460,464,468,472,476,480,484,488,492,496,500,504,508,512,516,520,524,528,532,536,539,540,541,542,544,545,546,549,550,553,554,557,558,560,561,562,564,567,568,569,572,573,576,577,580,581,583,586,587,588,591,592,595,596,598,599,601,602,603,605,608,609,610,612,615,616,617,620,621,624,625,628,629,631,632,633,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,655,656,657,658,802,883,884,885,888,890,89

         Legal Fund Name Including Umbrella Fund Code Citi Code          ISIN  \
0                                      C 11     00F9A      009A  AT0000721402   
1                                      C 11     00F9A      009A  AT0000721402   
2                                  R-VIP 50     00F9G      009G  AT0000A0FA24   
3                                  R-VIP 50     00F9G      009G  AT0000A0FA24   
4                                 R-VIP 100     00F9I      009I  AT0000A0F9X2   
5                                 R-VIP 100     00F9I      009I  AT0000A0F9X2   
6                          Raiffeisen R 135     00F9K      009K  AT0000691365   
7                          Raiffeisen R 135     00F9K      009K  AT0000691365   
8  JPMorgan Funds - Europe Equity Plus Fund     AEFD9      00A2  LU0289214545   
9  JPMorgan Funds - Europe Equity Plus Fund     AEFD9      00A2  LU0289214545   

      WKN  Apir Code    SEDOL  As Of Date  ESG Score  ESG Rating  ...  \
0  A0J4QW        NaN      NaN      

In [7]:
with open("colunas_especificas.txt", 'r') as fin:
    cols = fin.readlines()
    cols_especificas = []
    for c in cols:
        cols_especificas.append(c.replace("\n",""))


cols_especificas

['Legal Fund Name Including Umbrella', 'Fund Code', 'Citi Code', 'ISIN', 'WKN', '20000_Financial_Instrument_Identifying_Data', 'EPT Portfolio Name', 'EPT Portfolio or Share Class Currency', 'EPT PRIIPs KID Publication Date', 'EPT Reference Language', 'EPT Valuation Frequency', 'EPT Summary Risk Indicator', 'EPT Is SRI Adjusted', 'EPT Market Risk Measure', 'EPT Credit Risk Measure', 'EPT Recommended Holding Period', 'EPT Number Of Observed Returns', 'EPT Reference Invested Amount', 'EPT Ongoing Costs Management Fees and Other Administrative or Operating Costs', 'EPT Portfolio Transaction Costs', 'EPT Target Market Retail Investor Type', 'EPT Investment Objective', 'EPT Risk Narrative', 'EPT Other Risk Narrative', 'EPT Investment Option', 'Portfolio Turnover Ratio', 'Portfolio Turnover Ratio Date', 'FE Group Name', 'Unit Status', 'Company Contact Address', 'Company Contact E-Mail', 'Company Contact Post Code', 'Company Contact Telephone', 'Company Contact Web Address', 'FE Fund Name', 'S

In [22]:
# Função para comparar mudanças em chunks

# Função para comparar mudanças em chunks
def compare_chunks(file1, file2, identifiers, columns_to_compare):
    changes = []

    # Ler os arquivos CSV em chunks
    reader1 = pd.read_csv(file1, chunksize=chunksize)
    reader2 = pd.read_csv(file2, chunksize=chunksize)
    #df = pd.concat(reader1, reader2,ignore_index=True)
    # Iterar sobre os chunks de ambos os arquivos
    for chunk1, chunk2 in zip(reader1, reader2):
        # Converta tipos para garantir que são consistentes
        for identifier in identifiers:
            chunk1[identifier] = chunk1[identifier].astype(str)
            chunk2[identifier] = chunk2[identifier].astype(str)

        # Juntar os DataFrames com base nos identificadores
        merged_df = pd.merge(chunk1, chunk2, on=identifiers, suffixes=('_old', '_new'))

        # Analisar mudanças nas colunas específicas
        for column in columns_to_compare:
            old_column = f"{column}_old"
            new_column = f"{column}_new"

            # Verifique se as colunas existem
            if old_column not in merged_df.columns or new_column not in merged_df.columns:
                continue

            # Identifica onde há mudanças
            mask = merged_df[old_column] != merged_df[new_column]
            if mask.any():
                # Adiciona as mudanças ao resultado
                changes.append(merged_df.loc[mask, [*identifiers, old_column, new_column]])

    # Retorna todas as mudanças em um DataFrame
    return pd.concat(changes, ignore_index=True) if changes else pd.DataFrame()

# Defina os arquivos CSV e as colunas
file1 = 'Quali_20240601.csv'
file2 = 'Quali_20240602.csv'
identifiers = ['ISIN', 'Citi Code', 'EPT Reference Language']  # Certifique-se de que os nomes estejam corretos
columns_to_compare = cols_especificas  # As colunas que você deseja comparar

# Chamar a função para comparar os chunks
changes_df = compare_chunks(file1, file2, identifiers, columns_to_compare)

# Exibir mudanças
if not changes_df.empty:
    print("Mudanças encontradas:")
    print(changes_df)
else:
    print("Nenhuma mudança encontrada.")

# Salvar mudanças em um novo CSV, se necessário
# changes_df.to_csv('mudancas.csv', index=False)



# Defina os arquivos CSV e as colunas
file1 = 'Quali_20240601.csv'
file2 = 'Quali_20240602.csv'
identifiers = ['ISIN', 'Citi Code', 'EPT Reference Language']
columns_to_compare = cols_especificas  

# Chamar a função para comparar os chunks
changes_df = compare_chunks(file1, file2, identifiers, columns_to_compare)

# Exibir mudanças
if not changes_df.empty:
    print("Mudanças encontradas:")
    print(changes_df)
else:
    print("Nenhuma mudança encontrada.")

# Salvar mudanças em um novo CSV, se necessário
changes_df.to_csv('mudancas.csv', index=False)


C:\Users\Vini\AppData\Local\Temp\ipykernel_13644\3270697294.py:12: DtypeWarning: Columns (5,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,103,104,107,111,112,113,114,115,116,117,118,120,121,122,124,125,127,138,139,140,142,149,158,159,160,163,164,173,174,175,176,177,178,181,182,184,188,192,196,200,204,208,212,216,220,224,228,232,236,240,244,248,252,256,260,264,268,272,276,280,284,288,292,296,300,304,308,312,316,320,324,328,332,336,340,344,348,352,356,360,364,368,372,376,380,384,388,392,396,400,404,408,412,416,420,424,428,432,436,440,444,448,452,456,460,464,468,472,476,480,484,488,492,496,500,504,508,512,516,520,524,528,532,536,539,540,541,542,544,545,546,549,550,553,554,557,558,560,561,562,564,567,568,569,572,573,576,577,580,581,583,586,587,588,591,592,595,596,598,599,601,602,603,605,608,609,610,612,615,616,617,620,621,624,625,628,629,631,632,633,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,655,656,657,658,802,883,884,885,888,890,8

KeyboardInterrupt: 